In [2]:
import pandas as pd
import numpy as np

In [3]:
train = pd.read_pickle('./train_data_with_ind.pkl')
val = pd.read_pickle('./val_data_with_ind.pkl')
# test = pd.read_pickle('./test_data_with_ind.pkl')

In [4]:
target = pd.read_pickle('./target.pkl')
target.set_index('DATE', drop=True, inplace=True)

In [5]:
train_target = target.loc['19570101':'19750101']

In [6]:
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor

In [7]:
train.fillna(0, inplace=True)

In [8]:
val.fillna(0, inplace=True)

In [28]:
train_target

,permno,RET
DATE,,
19570131,10006,0.064378
19570131,10014,0.095238
19570131,10022,0.102041
19570131,10030,-0.047091
19570131,10057,-0.090062
...,...,...
19741231,84794,-0.233766
19741231,84903,-0.071429
19741231,85682,-0.210526


In [10]:
val_target = target.loc['19750101':'19870101']

In [92]:
for k, i in tqdm(enumerate(range(12, len(val.index.get_level_values(0).unique()[:72]), 12))):
    date = val.index.get_level_values(0).unique()[i]
    train1 = pd.concat([train, val.loc[:date]], axis=0)
    train_target1 = pd.concat([train_target, val_target.loc[:date]], axis=0)
    mod = RandomForestRegressor(n_estimators=20, n_jobs=-1)
    mod.fit(np.array(train1), np.array(train_target1.loc[:,'RET']))
    
# 保存模型
    with open('rf'+str(k)+'.pkl', 'wb') as file:
        pickle.dump(mod, file)

5it [4:12:21, 3028.31s/it]


In [17]:
from tqdm import tqdm
from sklearn.ensemble import GradientBoostingRegressor
import pickle
for k, i in tqdm(enumerate(range(12, len(val.index.get_level_values(0).unique()[:72]), 12))):
    date = val.index.get_level_values(0).unique()[i]
    train1 = pd.concat([train, val.loc[:date]], axis=0)
    train_target1 = pd.concat([train_target, val_target.loc[:date]], axis=0)
    mod = GradientBoostingRegressor(loss='huber', n_estimators=20, learning_rate=0.1, random_state=123)
    mod.fit(np.array(train1), np.array(train_target1.loc[:,'RET']))
    
# 保存模型
    with open('gb'+str(k)+'.pkl', 'wb') as file:
        pickle.dump(mod, file)

4it [2:00:11, 1802.88s/it]


In [ ]:
test = pd.read_pickle('./test_data_with_ind.pkl')

In [ ]:
test_target = target.loc['19870101':'20170101']

In [ ]:
import pickle
r_square_rf = []
from tqdm import trange
for i in trange(5):
    
    with open('rf'+str(i)+'.pkl', 'rb') as file:
        rf = pickle.load(file)
    test_pre = rf.predict(np.array(test.loc[test.index.get_level_values(0).unique()[i*12:180]]))
    test_target = test_target.loc[test.index.get_level_values(0).unique()[i*12:180]]
    r_square_rf.append(1-sum((test_pre - np.array(test_target.iloc[:, 1])) ** 2) / sum(np.array(test_target.iloc[:, 1]) ** 2))


In [ ]:
import matplotlib.pyplot as plt
rf_importance = pd.DataFrame(index=test.columns, columns=['importance'])
rf_importance.importance = list(rf.feature_importances_)
rf_importance.sort_values(by='importance',ascending=False,inplace=True)
rf_importance.index = [str(index) for index in rf_importance.index]
plt.barh(rf_importance.index[:20], rf_importance.importance.iloc[:20])

In [ ]:
r_square_gb = []
for i in trange(5):
    
    with open('gb'+str(i)+'.pkl', 'rb') as file:
        gb = pickle.load(file)
    test_pre = gb.predict(np.array(test.loc[test.index.get_level_values(0).unique()[i*12:180]]))
    test_target = test_target.loc[test.index.get_level_values(0).unique()[i*12:180]]
    r_square_gb.append(1-sum((test_pre - np.array(test_target.iloc[:, 1])) ** 2) / sum(np.array(test_target.iloc[:, 1]) ** 2))


In [ ]:
import matplotlib.pyplot as plt
gb_importance = pd.DataFrame(index=test.columns, columns=['importance'])
gb_importance.importance = list(gb.feature_importances_)
gb_importance.sort_values(by='importance',ascending=False,inplace=True)
gb_importance.index = [str(index) for index in gb_importance.index]
plt.barh(gb_importance.index[:20], gb_importance.importance.iloc[:20])